In [1]:
n=2 # number of attributes
s=2 # number of classes
t=Vector{Int64}(undef,s) # sample sizes for traint set
Y=Vector{Matrix{Float64}}(undef,s) # input data
Y_train=Vector{Matrix{Float64}}(undef,s) # traint set
Y_test=Vector{Matrix{Float64}}(undef,s) # test set
ratio=0.8 # ratio of train set to test set

for k=1:s 
    # take random samples
    Y[k]=Matrix{Float64}(undef,20,n)
    for j=1:20
        randx=2*rand(Float64,n).-1
        randx=0.5*rand(Float64)*randx./sqrt(sum(randx.^2))
        Y[k][j,:]=randx+(2*k-3)*[0.25;0]
    end
    
    t[k]=ceil(Int64,ratio*size(Y[k],1))
    Y_train[k]=Y[k][1:t[k],:]
    Y_test[k]=Y[k][(t[k]+1):end,:]
end

d=Vector{Int64}(undef,s) # degrees of polynomial estimations

include("../src/ClassificationModels2.jl")
using .ClassificationModels2

Lambda=Vector{Function}(undef,s) # polynomial approximations of the indicator function of classes

r=1 # radious of the ball centered at origin containning all samples
c=Inf

for k=1:s
    println("Class ",k)
    println()
    d[k]=2
    
    # train a model
    Lambda[k]=ClassificationModels2.model_volume(n,Y_train[k],t[k],r,d[k],
                                            ball_cons=true,bound=Inf,delt=1+1/c,
                                            bound_coeff=c,Stokes_constraint=false);
    println("------------")
end

function classifier(y)
    return findmax([Lambda[k](y) for k=1:s])[2]
end

predict=Vector{Vector{Int64}}(undef,s) # prediction
numcor=Vector{Int64}(undef,s) # number of corrections

for k=1:s
    predict[k]=[classifier(Y_test[k][j,:]) for j in 1:size(Y_test[k],1)]
    numcor[k]=length(findall(u -> u == k, predict[k]))
end


accuracy=(sum(numcor))/(sum(size(Y_test[k],1) for k=1:s))

println("Accuracy on test set of volume computation: ",accuracy)

println()
println("==========================")
println()

k=2
c=Inf

eval_sep_pol=ClassificationModels2.model_SVM(n,Y_train,t,c,k,lamb=0.01)

function classifier2(y)
        if eval_sep_pol(y)>0
            return 1
        else
            return 2
        end
    end

predict=Vector{Vector{Int64}}(undef,s) # prediction
numcor=Vector{Int64}(undef,s) # number of corrections

for k=1:s
    predict[k]=[classifier2(Y_test[k][j,:]) for j in 1:size(Y_test[k],1)]
    numcor[k]=length(findall(u -> u == k, predict[k]))
end


accuracy=(sum(numcor))/(sum(size(Y_test[k],1) for k=1:s))

println("Accuracy on test set of support vector machine: ",accuracy)

Class 1

number of attributes: 2
sample size for traint set: 16
degree of polynomial estimation: 2
radius of the ball centered at the origin containing the samples: 1
maximal size of matrix variables: 6
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 16              
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number              

Accuracy on test set of support vector machine: 0.875
